In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import copy
import random
from mnist import MNIST

%matplotlib inline

Загрузка данных

In [2]:
x_train_dir = './data'

mndata = MNIST(x_train_dir)
mndata.gz = True

In [3]:
for i in range(2):
    print(mndata.display(X[i]))
    print(y[i])


............................
............................
............................
............................
............................
.....................@@.....
.............@@@@@@.@@......
........@@@@@@@@@@..........
........@@@@@@..@@..........
...........@@@..............
............@...............
............@...............
.............@..............
.............@@.............
..............@@@...........
................@@..........
.................@@.........
.................@@@........
.................@@@........
..............@@@@@.........
............@@@@@@..........
..........@@@@@.............
........@@@@@...............
......@@@@@@................
.....@@@@...................
............................
............................
............................
5

............................
............................
............................
............................
.................@..........
...............@@@@@........
..........

In [4]:
X = np.array(X)

In [5]:
record_num, features_num = X.shape
print(record_num, features_num)

60000 784


Нормировка, добавление bias-features.

In [6]:
def scale_data(X):
    X = X / 255
    
    return X

In [7]:
X = scale_data(X)

In [8]:
bias_f = np.ones((record_num, 1))
X = np.hstack((bias_f, X))

record_num, features_num = X.shape
print( record_num, features_num)

60000 785


In [9]:
record_num, features_num = X.shape
print( record_num, features_num)

60000 785


In [10]:
def dot_product(features, weights):
    inner_sum = np.sum(features * weights)
        
    return inner_sum

Стохастический градиент, градиентный спуск и функция предсказания.

In [11]:
def init_weights(features_num, a, b):
    return np.array(a + (b - a) * np.random.random(features_num))

In [12]:
def num_gradient_st(loss_fun, predict_fun, X, y, model_weights, w_delta=0.01):
    i = np.random.randint(0, len(X) - 1)
    current_loss = loss_fun(predict_fun(X[i], model_weights), y[i])
    weights_delta = model_weights[:]
    grad = []
    
    for coord in range(len(model_weights)):
        weights_delta[coord] += w_delta
        delta_loss = loss_fun(predict_fun(X[i], weights_delta), y[i])
        deriv = (delta_loss - current_loss) / w_delta
        grad.append(deriv)
        weights_delta[coord] -= w_delta
        
    return np.array(grad)

In [13]:
def gradient_descent(loss_fun, predict_fun, X, y, initial_weights, learning_rate, iter_num, verbose=False):
    model_weights = initial_weights[:]
    
    for counter in range(iter_num):
        grad = num_gradient_st(loss_fun, predict_fun, X, y, model_weights)
        model_weights -= learning_rate * grad
        #loss = loss_fun(X, y, model_weights)
        
        if verbose:
            print("Iter: %i, loss_value: %f" % (counter, loss))
        
    return model_weights

In [14]:
def one_vs_all(y, value):
    y_new = []
    for i in range(len(y)):
        if y[i] == value:
            y_new.append(1)
        else:
            y_new.append(-1)
    return y_new

In [15]:
def sign(x):
    return 1 if x >= 0.0 else -1

In [16]:
def svm_predict(features, weights):
    score = dot_product(features, weights)
    
    return score

In [17]:
def my_hinge_loss(predicted_value, true_value):
    result = max(0.0, 1.0 - predicted_value * true_value)
        
    return result

In [18]:
def one_vs_all(y, value):
    y_new = []
    for i in range(len(y)):
        if y[i] == value:
            y_new.append(1)
        else:
            y_new.append(-1)
    return y_new

In [19]:
def find_10_weights(X, y_bin, learning_rate = 0.05, iter_num = 100):
    weights_vector = []
    for yi in y_bin:
        initial_weights = init_weights(features_num, -0.5, 0.5)
        optimal_weights_i = gradient_descent(my_hinge_loss, svm_predict, X, yi, initial_weights, learning_rate, iter_num)
        weights_vector.append(optimal_weights_i)
    return weights_vector  

In [20]:
def y_split(y):
    y_10 = []
    for i in range(10):
        yi = one_vs_all(y, i)
        y_10.append(yi)
    return np.array(y_10)
#один против всех для всех чисел

In [21]:
def score_fun(X, y_bin, weights_vector):
    error_all = []
    for ind, yi in enumerate(y_bin):
            error = 0 
            for i in range(len(X)):
                score = svm_predict(X[i], weights_vector[ind])
                if score >= 0:
                    predict_val = 1
                else:
                    predict_val = -1
                error += abs(yi[i]-predict_val)
            error_all.append(error/2)
    return error_all

Подсчет на всех данных.

In [22]:
y_binary = y_split(y)

In [23]:
weights_vector = []

In [24]:
weights_vector.append(np.array([1]))

In [25]:
%%time
opt_weights_vector = find_10_weights(X, y_binary, iter_num = 20000)

Wall time: 21min 51s


In [26]:
opt_weights_vector

[array([ -2.90540956e+00,  -4.03026335e-01,   3.74840572e-01,
         -7.63183505e-05,  -3.47158678e-01,  -6.70674486e-03,
         -3.62318798e-01,   4.07226847e-01,   2.38201212e-01,
          4.02645660e-01,  -3.55308836e-02,   1.12950447e-01,
          3.62523455e-01,  -2.18217482e-01,  -1.13514363e-01,
          2.76016430e-01,  -2.70840001e-01,   3.73501140e-03,
         -2.78644926e-01,  -1.57671963e-01,  -6.68779654e-02,
         -6.46065549e-02,  -2.26672846e-01,   2.74656923e-01,
          3.21027707e-01,   3.93086275e-01,  -1.22937728e-01,
         -7.72856751e-02,  -3.84512073e-01,   3.24030440e-01,
         -4.04989134e-01,  -1.75550987e-01,  -4.17584588e-01,
          2.66638693e-01,   2.01569689e-01,   2.27109412e-01,
         -4.34742490e-01,  -2.89792034e-01,  -3.34595885e-01,
         -3.27280256e-01,  -3.16947682e-02,   2.26956705e-01,
          7.50908148e-02,   1.86516656e-01,  -1.07336388e-01,
          3.18506904e-01,  -8.85985581e-02,   1.38742362e-01,
        

In [27]:
%%time
print(score_fun(X, y_binary, opt_weights_vector))

[572.0, 614.0, 1362.0, 1906.0, 1221.0, 1971.0, 900.0, 1207.0, 4293.0, 3704.0]
Wall time: 4.47 s


In [28]:
def last_predict(X, weights_vector):
    predict_y = []
    for elem in X:
        score = []
        for weights in weights_vector:
            score.append(np.sum(elem*weights))
        score = np.array(score)
        i, = np.where(score == np.max(score))
        predict_y.append(int(i))
    return predict_y

In [29]:
print(last_predict(X[:10], opt_weights_vector))

[5, 0, 4, 1, 9, 2, 1, 3, 1, 4]


In [30]:
np.save('model_weights.txt', opt_weights_vector)

In [31]:
print(y[:10])

array('B', [5, 0, 4, 1, 9, 2, 1, 3, 1, 4])


In [32]:
print(np.append(np.array([1, 2, 3]), np.array([2, 3, 4])))

[1 2 3 2 3 4]


In [33]:
predicted_values = last_predict(X, opt_weights_vector)

In [34]:
print(classification_report(np.array(y), predicted_values))

             precision    recall  f1-score   support

          0       0.94      0.96      0.95      5923
          1       0.97      0.92      0.94      6742
          2       0.92      0.87      0.89      5958
          3       0.86      0.87      0.87      6131
          4       0.89      0.92      0.90      5842
          5       0.89      0.77      0.83      5421
          6       0.92      0.95      0.94      5918
          7       0.87      0.94      0.90      6265
          8       0.71      0.91      0.80      5851
          9       0.92      0.70      0.80      5949

avg / total       0.89      0.88      0.88     60000



Разделение данных на train, valid и test, подсчет на них.

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [36]:
y_train_binary = y_split(y_train)

In [37]:
%%time
opt_weights_vector = find_10_weights(X_train, y_train_binary, iter_num = 20000)

Wall time: 21min 38s


In [38]:
 predicted_values = last_predict(X_val, opt_weights_vector)

In [39]:
print(classification_report(np.array(y_val), predicted_values))

             precision    recall  f1-score   support

          0       0.95      0.96      0.95      1158
          1       0.94      0.96      0.95      1373
          2       0.90      0.86      0.88      1194
          3       0.84      0.89      0.86      1237
          4       0.79      0.94      0.86      1157
          5       0.94      0.64      0.76      1110
          6       0.93      0.93      0.93      1187
          7       0.95      0.87      0.91      1265
          8       0.75      0.86      0.80      1172
          9       0.84      0.85      0.84      1147

avg / total       0.88      0.88      0.88     12000



In [40]:
X_test, y_test = mndata.load_testing()

In [41]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [42]:
X_test = scale_data(X_test)

In [43]:
record_num, features_num = X_test.shape
bias_f = np.ones((record_num, 1))
X_test = np.hstack((bias_f, X_test))

record_num, features_num = X_test.shape
print( record_num, features_num)

10000 785


In [44]:
predicted_values_test = last_predict(X_test, opt_weights_vector)

In [45]:
print(classification_report(np.array(y_test), predicted_values_test))

             precision    recall  f1-score   support

          0       0.96      0.96      0.96       980
          1       0.96      0.96      0.96      1135
          2       0.93      0.85      0.89      1032
          3       0.84      0.92      0.88      1010
          4       0.82      0.94      0.87       982
          5       0.95      0.68      0.79       892
          6       0.93      0.95      0.94       958
          7       0.94      0.87      0.91      1028
          8       0.77      0.90      0.83       974
          9       0.86      0.85      0.85      1009

avg / total       0.90      0.89      0.89     10000



In [46]:
np.save('data\model_weights.npy', opt_weights_vector)